In [1]:
#! pip install boruta

In [2]:
# Cargo Librerías
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer  

# Levanto datos
df = pd.read_csv("/buckets/b1/datasets/competencia_03.csv.gz", compression='gzip')

# Elimino meses con clase ternaria nula
df = df[(df['foto_mes']!= 202108)]
df = df[(df['foto_mes']!= 202109)]

# Binarizo el target
df['clase_ternaria'] = df['clase_ternaria'].isin(['BAJA+2']).astype(int)

# Separo en X, y
X = df.drop(columns=["clase_ternaria"])
y = df[["clase_ternaria"]]

# Imputo faltantes porque no me acepta NA el borutapy
imputer = SimpleImputer(strategy='mean')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)
 
# Defino parámetros del modelo
rf_param = {
    'n_estimators': 500,  
    'min_samples_split': 250,
    'min_samples_leaf': 100,
    'max_depth': 14,
    'max_features': 'auto', 
    'random_state': 880001 
}

# Creo el modelo RF
rf = RandomForestClassifier(**rf_param)

# Instancio el Borutapy
feat_selector = BorutaPy(
    verbose=2,
    estimator=rf,
    n_estimators='auto',
    max_iter=153
)

# Entreno Boruta
feat_selector.fit(X_imputed.values, y.values.ravel())

columnas_seleccionadas = X.columns[feat_selector.support_]
print(columnas_seleccionadas)